# 序列去冗余

In [2]:
#-*-coding:utf-8-*-

import json
import pandas as pd
import random
import pickle
from collections import defaultdict


def get_dic(file):
    with open(file,'r')as f1:
        lines=f1.readlines()
    dic={}
    for line in lines:
        if '>' in line:
            epi=line.strip()
            dic[epi]=''
        else:
            dic[epi]+=line.strip()
    return dic
dic=get_dic('/home/dell-none/jupyter_notebook_code/covid19/all_seq.fasta')

id_list=list(dic.keys())

dic2={}
for v in dic.values():
    if v not in dic2.keys():
        dic2[v]=[]
print(len(dic2.keys()))
for k1 in dic2.keys():
    for k,v in dic.items():
        if v==k1:
            dic2[k1].append(k)
            
for i in dic1.values():
    ### 保留采样最早的序列
    
    kept=[]
    
    year=[int(i.split('_')[1].split('-')[0]) for j in i]
    month=[int(i.split('_')[1].split('-')[1]) for j in i]
    day=[int(i.split('_')[1].split('-')[1]) for j in i]
    
    if len(set(year))!=1:
        kept.append([j  for j in i if str(min(year)) in j ][0])
        
    elif len(set(year))!=1:
        kept.append([j for j in i if str(min(month)) in j ][0])
    
    elif len(set(day))!=1:
        
        kept.append([j for j in i if str(min(day)) in j ][0])
        
    else:
        
        kept.append(i[0])
fileter_dic={}     
for k,v in dic.items():
    
    if k in kept:
        
        fileter_dic[k]=v
def write_file(file,dict):
    with open(file,'w')as f2:
        for k1,v1 in dict.items():
            f2.write(k1+'\n'+v1+'\n')

write_file('/home/dell-none/jupyter_notebook_code/covid19/orign.fasta',fileter_dic)

FileNotFoundError: [Errno 2] No such file or directory: '/home/dell-none/jupyter_notebook_code/covid19/all_seq.fasta'

# 序列依据时间和地区采样

In [18]:
import json
import pandas as pd
import random
import numpy as np


###读取seq信息文件
df1=pd.read_excel('/home/dell-none/jupyter_notebook_code/covid19/covid19_info/all_info.xlsx')

###按时间和地区分组
group=df1.groupby(['region','collection_time']).size()

###
sample_list=[]
for (k1,k2), group in df1.groupby(['region','collection_time']):
    if len(group)<10:
        sample_list.append(group)
    else:

        n=group.sample(10)

        sample_list.append(n)
###合并df，写入文件
sample_seq_info=pd.concat(sample_list,axis=0).reset_index(drop=True)
sample_seq_info.to_csv('sample_seq_info.csv',index=False,sep=',')

seq_id=['>'+i for i in list(sample_seq_info['EPI_info'])]


###设置工作路径
os.chdir('/home/dell-none/jupyter_notebook_code/covid19')

###读取fasta文件到dic中
def get_dic(filename):
    dic = {}
    with open(filename, 'r')as f1:
        lines = f1.readlines()
    for line in lines:
        if '>' in line:
            epi = line.rstrip('\n')

            dic[epi] = ''
        else:
            dic[epi] += line.strip('\n')
    return dic

dic=get_dic("orign.fasta")


###将dic中的序列写入文件中
def write_file(file,dict):
    with open(file,'w')as f2:
        for k1,v1 in dict.items():
            f2.write(k1+'\n'+v1+'\n')

###写入采样序列fasta文件
sample_dic={}

for k,v in dic.items():
    if k in seq_id:
        sample_dic[k]=v
    if k=='>ref_seq_id':###此处填写seq_id的名称
        sample_dic[k]=v
        
write_file('sampled(l).fasta',sample_dic)

# 模糊碱基处理

In [2]:
#-*-coding:utf-8-*-
import json
import numpy as np
from collections import Counter
import pandas as pd 
import os

In [4]:

nucleotide=['A','T','C','G']

###替换每条序列中的模糊碱基为gap
def get_replace(seq):
    seq1=''
    for i in seq:
        if i in nucleotide:
            seq1+=i
        else:
            seq1+='-'
    return seq1.upper()



###指定需要读取的fasta文件
dic1=get_dic('sampled(l).fasta')


###运行get_replace函数，进行循环模糊碱基替换
def get_newdic(dict):
    dic={}
    for k,v in dict.items():
        dic[k]=get_replace(v)
    return dic
dic=get_newdic(dic1)

###输出序列中最长序列的长度
lenth=max([len(v) for v in dic.values()])

###删除gap大于400bp的碱基
del_lst=[]
for k,v in dic.items():
    if v.count('-')>=400:
        del_lst.append(k)
for del_ in del_lst:
    dic.pop(del_)

###处理后的序列字典写入文件中    
write_file('final_covid_l.fasta',dic)

# 选取CDS串联

In [ ]:
'''ORF1ab: 1-21290;
S: 21298-25119;
ORF3a: 25128-25955;
E: 25980-26207;
M: 26258-26926;
ORF6: 26937-27122;
ORF7: 27129-27622
ORF8: 27629-27994
N: 28009-29268
ORF10: 29293-29409
已去除终止密码子'''

###设置选取的cds字典
ORF1a={};ORF1b={};ORF1ab={};S={};ORF3a={};E={};M={};ORF6={};ORF7={};ORF8={}
N={};ORF10={};all_dic={}

###序列切片，各cds存入字典中
for k,v in dic.items():
    ORF1a[k]=v[:13203]
    ORF1b[k]=v[13202:]
    ORF1ab[k]=v[:21287]
    S[k]=v[21297:25116]
    ORF3a[k]=v[25127:25952]
    E[k]=v[25979:26204]
    M[k]=v[26257:26923]
    ORF6[k]=v[26936:27119]
    ORF7[k]=v[27128:27619]
    ORF8[k]=v[27628:27991]
    N[k]=v[28008:29265]
    ORF10[k]=v[29292:29406]
    all_dic[k]=all_

###cds序列写入文件中    
write_file('ORF1a_l.fasta',ORF1ab)
write_file('ORF1b_l.fasta',ORF1ab)
write_file('ORF1ab_l.fasta',ORF1ab)
write_file('S_l.fasta',S)
write_file('ORF3a_l.fasta',ORF3a)
write_file('E_l.fasta',E)
write_file('M_l.fasta',M)
write_file('ORF6_l.fasta',ORF6)
write_file('ORF7_l.fasta',ORF7)
write_file('ORF8_l.fasta',ORF8)
write_file('N_l.fasta',N)
write_file('ORF10_l.fasta',ORF10)
write_file('use_fasta_l.fasta',all_dic)

# 获取距离矩阵

In [ ]:
from tqdm import tqdm
from multiprocessing import Pool

dic1 = get_dic('cds.fasta')
lst1 = list(dic1.keys())

###序列转化成矩阵
result = []
for key in lst1:
    result.append(list(dic1[key]))
array_ = np.array(result)
del result

### 计算序列相差距离
def dis(index_):
    return np.sum(array_[index_] != array_[index_ + 1:], axis=-1)

###并行运算并写入json文件
if __name__ == '__main__':
    ### 并行计算距离
    with Pool(4) as p, open("covid19_cds_dis_l.json", 'w', encoding='utf-8') as f:
        pool_iter = p.imap(dis, range(len(lst1) - 1))
        for i, diss in enumerate(tqdm(pool_iter, total=len(lst1) - 1, desc="processing")):
            j = i + 1

            for dis_ in diss:
                data = [lst1[i], lst1[j], int(dis_)]
                j+=1
                f.write(json.dumps(data, ensure_ascii=False) + '\n')

###注意清除内存，不然会爆掉
del data

# 填补gap

In [45]:
from tqdm import tqdm
dis={}

with open("/home/sunqr/Desktop/covid19_cds_dis_l.json", 'r') as f:
        
        for line in tqdm(f):
            data=json.loads(line)
            name=data[0]+'|'+data[1]
            dis[name]=data[2]
                

def return_top_hamming(dis,k):
    relatdic={}
    for k1,v1 in dis.items():
#         print(k1)
        if k1.split('|')[0]==k:
#             print(k1.split('|')[0])
            relatdic[k1]=v1
    sort_rel=sorted(relatdic.items(), key = lambda relatdic:relatdic[1])[:10]
    
    top10=[i[0].split('|')[1] for i in sort_rel]
   
    
    return  top10               

14303226it [01:13, 194356.88it/s]


In [ ]:
### 获取每个位点上top10最高频碱基

def highfre_site(seq):
    
    array=np.array(seq)
    s=Counter(array).most_common(1)[0][0]###得到碱基个数最多的一个碱基
    if s!="-":
        pass
    else:
        s=Counter(array).most_common(2)[1][0]
    return s
high_site=[]

f_dic=get_dic('use_fasta_l.fasta')

###### 获取top10位点构成序列
def get_top10_com(top10):
    need_dic={k:v for k,v in f_dic.items() if k in top10}
    lenth=max([len(v) for v in need_dic.values()])
    print(lenth)
    for i in range(lenth):

        seq=[]


        for k,v in need_dic.items():
            seq.append(v[i])

        s=highfre_site(seq)
        high_site.append(s)
        
    return high_site
    
### ###以最高频碱基更新序列，填补gap
ff_dic={}
for k,v in f_dic.items():
    top10=return_top_hamming(dis,k)
    high_site=get_top10_com(top10)
    v1=''
    for i in range(lenth):
        if v[i]=='-':
            
            
            
            v1+=high_site[i]
        else:
            v1+=v[i]
    ff_dic[k]=v1

###填补后写入文件
write_file('use_fasta_l.fasta2',ff_dic)

#  序列去冗余

In [11]:
delete_dic=get_dic('use_fasta_l.fasta2')

###序列与名称倒置
r_dic={}
for v in delete_dic.values():
    if v not in r_dic.keys():
        r_dic[v]=[]
print(len(r_dic.keys()))

###获得唯一的序列字典
for k1 in r_dic.keys():
    for k,v in delete_dic.items():
        if v==k1:
            r_dic[k1].append(k)
print(len(r_dic))

###序列与名称再倒置回来，完成去冗余
p_dic={}
for k,v in r_dic.items():
    p_dic[v[0]]=k
    
del r_dic
del delete_dic

write_file('cds.fasta',p_dic)

5349
5349


# 计算突变频率

In [ ]:
dic2={}

###计算不同碱基转换类型次数
def get_mutation(s1,s2):
    for i in range(len(s1)):
        if s1[i]!= s2[i]:
            key=s1[i]+'-'+s2[i]
            if key in dic2.keys():
                dic2[key]+=1
            else:
                dic2[key]=1

###读取上步形成的json文件
with open("covid19_cds_dis_l.json", 'r') as f:
        for line in tqdm(f):
            data=json.loads(line)
            if data[2] == 1:
                s1=dic1[data[0]].upper()
                s2=dic1[data[1]].upper()
                get_mutation(s1,s2)


del s1
del s2

###转换成频率，保留三位小数
dic={}
for k,v in dic2.items():
    if k=='G-T' or k=='T-G':
        dic[('T','G')]=round((dic2['T-G']+dic2['G-T'])/t,3)
        dic[('G','T')]=round((dic2['T-G']+dic2['G-T'])/t,3)
    if k=='T-A' or k=='A-T':
        dic[('T','A')]=round((dic2['T-A']+dic2['A-T'])/t,3)
        dic[('A','T')]=round((dic2['T-A']+dic2['A-T'])/t,3)
    if k=='G-A' or k=='A-G':
        dic[('G','A')]=round((dic2['G-A']+dic2['A-G'])/t,3)
        dic[('A','G')]=round((dic2['G-A']+dic2['A-G'])/t,3)
    if k=='T-C' or k=='C-T':
        dic[('T','C')]=round((dic2['T-C']+dic2['C-T'])/t,3)
        dic[('C','T')]=round((dic2['T-C']+dic2['C-T'])/t,3)
    if k=='A-C' or k=='C-A':
        dic[('A','C')]=round((dic2['A-C']+dic2['C-A'])/t,3)
        dic[('C','A')]=round((dic2['A-C']+dic2['C-A'])/t,3)
    if k=='G-C' or k=='C-G':
        dic[('G','C')]=round((dic2['G-C']+dic2['C-G'])/t,3)
        dic[('C','G')]=round((dic2['G-C']+dic2['C-G'])/t,3)
        
###写入文件中留存
with open('mutation_cds_matrix_l.json','w')as f1:
    json.dump(dic,f1)

# 获取序列间pro_matrix

In [ ]:
from multiprocessing import Pool, cpu_count

dic1=dic
del dic

keys = []
seqs = []
for key, value in dic1.items():
    keys.append(key)
    seqs.append(list(value))

seqs = np.array(seqs)

###enumerate函数：遍历一个列表的值并且返回每个值对应的下标
def func(i):
    ###计算每对序列间转换概率并记录最大值
    seq = seqs[i]
    indexs = seq != seqs  # 计算不同元素的位置
    max_prob = 0
    # max_key = ''
    for j, index in enumerate(indexs):
        if i == j:
            continue
        index = np.where(index)  # 找到不同的下标
        probs = [dic[(c1, c2)] for c1, c2 in zip(seq[index], seqs[j][index])]
        prob = 1
        for p in probs:
            prob *= p
        if prob >= max_prob:
            max_prob=prob
    return max_prob
        # {'key': max_key, 'prob': max_prob}


if __name__ == '__main__':
### 并行计算所有序列的概率最大值
    threads = 6
    threads = min(threads, cpu_count())
    with Pool(threads) as p:
        results = list(
            tqdm(
                p.imap(func, range(len(keys)), chunksize=32),
                total=len(keys),
                desc="processing"
            )
        )
with open('prob_cds_results_l.json','w')as f2:
    json.dump(results,f2)


###计算每条序列间转换的概率
def get_prob(s1,s2):
    p=1
    for i in range(len(s1)):
        if (s1[i],s2[i]) in dic.keys():
            p=p*dic[(s1[i],s2[i])]
    return p


# def func(i):
#     prob = []
#     for j in range(i+1,len(lst)):
#             pro=get_prob(dic1[lst[i]],dic1[lst[j]])
#     prob.append((i,j,pro))
#     return prob
keys = []
seqs = []
for key, value in dic1.items():
    keys.append(key)
    seqs.append(list(value))

seqs = np.array(seqs)


# ##enumerate函数：遍历一个列表的值并且返回每个值对应的下标
def func(i):
    seq = seqs[i]
    indexs = seq != seqs  # 计算不同元素的位置

    pro_matrix = []
    for j, index in enumerate(indexs):
        if i == j:
            continue
        index = np.where(index)  # 找到不同的下标
        probs = [dic[(c1, c2)] for c1, c2 in zip(seq[index], seqs[j][index])]
        prob = 1
        for p in probs:
            prob *= p
        pro_matrix.append([keys[i],keys[j],prob])
    return pro_matrix


###并行计算两两序列间转换的概率并写入json文件
if __name__ == '__main__':

    threads = 11
    threads = min(threads, cpu_count())
    with Pool(threads) as p,open('pro_cds_matrix_l.json','w',encoding='utf-8')as f2:
        pro_iter = tqdm(p.imap(func, range(len(keys)),chunksize=32),total=len(keys))
        for dis in pro_iter:
            f2.write(json.dumps(dis, ensure_ascii=False) + '\n')